In [24]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [6]:
embedding=OllamaEmbeddings(model="nomic-embed-text")

In [21]:
vectordb = Chroma(persist_directory="./chroma_db", embedding_function=embedding)

In [14]:
prompt = hub.pull("rlm/rag-prompt")

In [18]:
llm = Ollama(model="codellama",verbose=True,
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),)

In [22]:
# qa_chain = RetrievalQA.from_chain_type(
#         llm,
#         retriever=vectordb.as_retriever(),
#         chain_type_kwargs={"prompt": prompt},
#         return_source_documents=True,
#     )

In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [26]:
rag_chain = (
    {"context": vectordb.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
rag_chain.invoke("""
                 can you explain the following code to me?
type RoomMenuProps = {
	rid: string;
	unread?: boolean;
	threadUnread?: boolean;
	alert?: boolean;
	roomOpen?: boolean;
	type: RoomType;
	cl?: boolean;
	name?: string;
	hideDefaultOptions: boolean;
};
""")

In [ ]:
docs = db.similarity_search("""type RoomMenuProps = {
	rid: string;
	unread?: boolean;
	threadUnread?: boolean;
	alert?: boolean;
	roomOpen?: boolean;
	type: RoomType;
	cl?: boolean;
	name?: string;
	hideDefaultOptions: boolean;
};""",k=1)